In [68]:
pip install haversine

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: haversine in c:\users\sohyeun\appdata\local\programs\python\python38\lib\site-packages (2.8.1)




[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [69]:
import pandas as pd
import numpy as np
from pyproj import Proj, transform
from haversine import haversine

In [70]:
center = pd.read_csv("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\헥사곤_중심.csv")
park = pd.read_csv("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\데이터\\주차장\\Beam_스윙_씽씽_주차장.csv", encoding='cp949')
bus = pd.read_csv("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\데이터\\유동인구\\서울시 버스정류소 위치정보(광진구).csv", encoding='cp949')
subway = pd.read_excel("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\데이터\\유동인구\\지하철역_위치(수정).xlsx")
tow = pd.read_csv("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\데이터\\견인\\견인지역_2023.csv", encoding='cp949')

In [71]:
center['X'] = (center['left'] + center['right']) / 2
center['Y'] = (center['bottom'] + center['top']) / 2
center.drop(columns=['left', 'right', 'bottom', 'top', '최종_레이블_결과(2)_행정동', '최종_레이블_결과(2)_클러스터링', 
                '최종_레이블_결과(2)_유동인구', '최종_레이블_결과(2)_견인횟수', '최종_레이블_결과(2)_유동인구_레이블', '최종_레이블_결과(2)_견인횟수_레이블'], inplace=True)
center.dropna(inplace=True)
center.reset_index(drop=True, inplace=True)
center.head()

,id,row_index,col_index,최종_레이블_결과(2)_유동인구+견인횟수,X,Y
0,3110,31,54,4.0,965523.060907,1.949635e+06
1,3111,32,54,4.0,965523.060907,1.949535e+06
2,3112,33,54,4.0,965523.060907,1.949435e+06
3,2940,32,51,4.0,965263.253286,1.949485e+06
4,2941,33,51,4.0,965263.253286,1.949385e+06


In [72]:
# 원본 좌표 시스템 정의 (UTM 좌표)
original_crs = Proj(init='epsg:5179')  # UTM-K 좌표계 (한국)

# 변환할 좌표 시스템 정의 (WGS84, 위경도)
target_crs = Proj(init='epsg:4326')  # WGS84 좌표계 (위경도)

# 좌표 변환 함수
def convert_coordinates(x, y):
    lon, lat = transform(original_crs, target_crs, x, y)
    return lat, lon

# 'X'와 'Y' 좌표를 UTM-K 좌표계에서 WGS84 좌표계로 변환
center['경도'], center['위도'] = zip(*center.apply(lambda row: convert_coordinates(row['X'], row['Y']), axis=1))
center.drop(columns=['row_index', 'col_index', 'X', 'Y'], inplace=True)
center.rename(columns={'최종_레이블_결과(2)_유동인구+견인횟수': '유동인구+견인횟수'}, inplace=True)
center.rename(columns={'위도': '경도', '경도': '위도'}, inplace=True)
center.head()

c:\Users\SOHYEUN\AppData\Local\Programs\Python\Python38\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\SOHYEUN\AppData\Local\Programs\Python\Python38\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\SOHYEUN\AppData\Local\Temp\ipykernel_3236\3619445213.py:9: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/g

,id,유동인구+견인횟수,위도,경도
0,3110,4.0,37.545402,127.109710
1,3111,4.0,37.544500,127.109715
2,3112,4.0,37.543599,127.109719
3,2940,4.0,37.544040,127.106776
4,2941,4.0,37.543139,127.106781


In [73]:
bus.head()

,노드 ID,정류소번호,정류소명,위도,경도,정류소 타입
0,104900058,5504,정보도서관,37.550249,127.110462,마을버스
1,104000281,5745,워커힐호텔비스타워커힐,37.554242,127.110265,일반차로
2,104900063,5744,워커힐팰리스,37.550013,127.109672,마을버스
3,104900057,5509,한강호텔입구,37.548455,127.109452,마을버스
4,104900064,5742,한강호텔입구,37.548275,127.109315,마을버스


In [74]:
subway.head()

,호선명,역명,위도,경도
0,2호선,강변(동서울터미널),37.535095,127.094681
1,2호선,건대입구(2호선),37.540693,127.070230
2,7호선,건대입구(7호선),37.540693,127.070230
3,5호선,광나루(장신대),37.545303,127.103570
4,5호선,군자(능동)(5호선),37.557121,127.079542


In [75]:
park.head()

,동명,주차장명,위도,경도,업체
0,광장동,광장삼성2차아파트 나동,37.542413,127.101819,Beam
1,광장동,대교 경정비 건너편,37.541566,127.096836,Beam
2,광장동,대교 경정비,37.541343,127.096574,Beam
3,구의1동,구의사거리,37.545064,127.085321,Beam
4,구의1동,커피 BAY,37.542201,127.083972,Beam


In [76]:
tow.head()

,번호,구정보,신고일,조치일,주소,유형,위도,경도
0,1.0,광진구,2023-01-17,2023-01-17,서울특별시 광진구 중곡동 266-1,보도와 차도가 구분된 도로의 차도,37.565554,127.086574
1,2.0,광진구,2023-01-17,2023-01-17,서울특별시 광진구 광장동 551,보도와 차도가 구분된 도로의 차도,37.545976,127.105052
2,3.0,광진구,2023-01-18,2023-01-18,서울특별시 광진구 자양동 204-1,기타,37.536761,127.083275
3,4.0,광진구,2023-01-18,2023-01-18,서울특별시 광진구 구의동 546-4,보도 중앙,37.535699,127.095765
4,5.0,광진구,2023-01-18,2023-01-18,서울특별시 광진구 구의동 546-4,보도 중앙,37.535699,127.095765


In [77]:
center_points = np.array([list(i) for i in zip(center['위도'], center['경도'])])
bus_points = np.array([list(i) for i in zip(bus['위도'], bus['경도'])])
subway_points = np.array([list(i) for i in zip(subway['위도'], subway['경도'])])
park_points = np.array([list(i) for i in zip(park['위도'], park['경도'])])
tow_points = np.array([list(i) for i in zip(tow['위도'], tow['경도'])])

In [78]:
X = list(bus['경도']) + list(subway['경도']) + list(park['경도']) + list(tow['경도'])
Y = list(bus['위도']) + list(subway['위도']) + list(park['위도']) + list(tow['위도'])
points = np.array([list(i) for i in zip(X, Y)])
print(points.shape)
points[:2]

(2795, 2)


array([[127.1104622 ,  37.55024867],
       [127.1102655 ,  37.5542416 ]])

In [79]:
total_w = points.shape[0]
bus_w = bus_points.shape[0]
subway_w = subway_points.shape[0]
park_w = park_points.shape[0]
tow_w = tow_points.shape[0]

# 데이터 불균형 해소를 위한 가중치
m1 = (total_w-bus_w) / total_w
m2 = (total_w-subway_w) / total_w
m3 = (total_w-park_w) / total_w
m4 = (total_w-tow_w) / total_w

In [80]:
def pmedian(center_points, bus_points, subway_points, park_points, tow_points):
    
    # 버스
    havers1 = []
    for i in center_points:
        site1 = []
        for j in bus_points:
            site1.append(haversine(i,j)*1000)
        havers1.append(site1)

    location = list(center['id'])
    location1 = list(bus['정류소명'])

    havers_D1 = dict(zip(location, [dict(zip(location1, i)) for i in havers1]))
    D1 = pd.DataFrame(havers_D1)
    
    bus_min = list(D1.min(axis=1))

    # 최단거리인 경우 가중치 부여, 아니면 0
    idx = D1.index
    col = D1.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D1.loc[idx[i], col[j]] == bus_min[i]:
                D1.loc[idx[i], col[j]] = m1
            else:
                D1.loc[idx[i], col[j]] = 0
    
    
    # 지하철
    havers2 = []
    for i in center_points:
        site2 = []
        for j in subway_points:
            site2.append(haversine(i,j)*1000)
        havers2.append(site2)

    location2 = list(subway['역명'])

    havers_D2 = dict(zip(location, [dict(zip(location2, i)) for i in havers2]))
    D2 = pd.DataFrame(havers_D2)
    
    subway_min = list(D2.min(axis=1))

    idx = D2.index
    col = D2.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D2.loc[idx[i], col[j]] == subway_min[i]:
                D2.loc[idx[i], col[j]] = m2
            else:
                D2.loc[idx[i], col[j]] = 0
                
            
    # 주차장
    havers3 = []
    for i in center_points:
        site3 = []
        for j in park_points:
            site3.append(haversine(i,j)*1000)
        havers3.append(site3)

    location3 = list(park['주차장명'])

    havers_D3 = dict(zip(location, [dict(zip(location3, i)) for i in havers3]))
    D3 = pd.DataFrame(havers_D3)
    
    park_min = list(D3.min(axis=1))

    idx = D3.index
    col = D3.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D3.loc[idx[i], col[j]] == park_min[i]:
                D3.loc[idx[i], col[j]] = m3
            else:
                D3.loc[idx[i], col[j]] = 0


    # 견인위치
    havers4 = []
    for i in center_points:
        site4 = []
        for j in tow_points:
            site4.append(haversine(i,j)*1000)
        havers4.append(site2)

    location4 = list(tow['주소'])

    havers_D4 = dict(zip(location, [dict(zip(location4, i)) for i in havers4]))
    D4 = pd.DataFrame(havers_D4)
    
    tow_min = list(D4.min(axis=1))

    idx = D4.index
    col = D4.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D4.loc[idx[i], col[j]] == tow_min[i]:
                D4.loc[idx[i], col[j]] = m4
            else:
                D4.loc[idx[i], col[j]] = 0
                

    D_final = pd.concat([D1, D2, D3, D4])

    return D_final

In [81]:
D_final = pmedian(center_points, bus_points, subway_points, park_points, tow_points)

In [82]:
D_final

,3110,3111,3112,2940,2941,2942,2943,2937,2938,2939,...,219,164,165,166,160,161,162,163,159,3167
정보도서관,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
워커힐호텔비스타워커힐,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
워커힐팰리스,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.895886,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
한강호텔입구,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
워커힐아파트앞,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.895886,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
서울특별시 광진구 군자동 313,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,...,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157
서울특별시 광진구 구의동 243-66,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,...,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157
서울특별시 광진구 중곡동 280-4,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,...,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157
서울특별시 광진구 중곡동 273-1,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,...,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157,0.168157


In [84]:
result_df = pd.DataFrame(D_final.sum().sort_values(ascending=False), columns=['Weight'])

# index를 column으로 이동
result_df.reset_index(inplace=True)
result_df

,index,Weight
0,1841,6.888730
1,2880,6.888730
2,1348,5.272987
3,950,5.272987
4,1579,5.272987
...,...,...
1282,1611,1.513417
1283,1609,1.513417
1284,1608,1.513417
1285,1615,1.513417


In [85]:
result_df_20 = result_df[:20]
result_df_30 = result_df[:30]

In [86]:
result_df_20.to_csv("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\P-median_20.csv", index=False)
result_df_30.to_csv("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\P-median_30.csv", index=False)

In [87]:
result_df_20 = pd.read_csv("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\P-median_20.csv")
result_df_30 = pd.read_csv("C:\\Users\\SOHYEUN\\Desktop\\BOAZ_광진구\\광진구_최종\\P-median_30.csv")

In [88]:
result_df_20

,index,Weight
0,1841,6.888730
1,2880,6.888730
2,1348,5.272987
3,950,5.272987
4,1579,5.272987
5,1444,5.184973
6,663,5.184973
7,2995,5.096959
8,1842,5.096959
9,835,4.401431
